This is the second pratice code for the application of Double Machine Learning (DML) to estimate the effect of 401(k) elligiblity on retirement savings. 
The 401K data considered here is from the Survey of Income and Program Participation (SIPP) from the year 1991. We use the data taken from the application in Chernozhukov et al. (2018). This time we play around with different models and compare the results. 
401(k) plans are pension accounts sponsored by employers. A major challenge in assessing the impact of participating in 401(k) plans on accumulated savings arises from saver heterogeneity and the non-random nature of enrollment decisions. Individuals differ significantly in their preferences for saving, and it is reasonable to assume that those with stronger, unobserved preferences for saving are more inclined to participate in tax-advantaged retirement plans such as 401(k)s. Consequently, these individuals likely have higher accumulated assets regardless of participation. Failure to account for this saver heterogeneity and the endogenous nature of participation decisions leads to upward-biased estimates, overstating the actual savings effects attributable to 401(k) participation.

To address this bias, it can be argued that eligibility for enrolling in a 401(k) plan can be considered exogenous once certain observable factors, especially income, are controlled for. This argument rests on the premise that when 401(k) plans were initially introduced, employment decisions were generally driven by income and other job characteristics, rather than by the availability of a 401(k). Thus, after conditioning on income and similar observables, eligibility for a 401(k) is less likely to be correlated with unobserved preferences for saving.- source:https://docs.doubleml.org/dev/examples/py_double_ml_pension.html

We use net financial assets (net_tfa) as the outcome variable. The treatment variable is 401(k) elligibility (e401). The covariates are age, income, education, marital status and other worker characteristics such as family size, dual-earner status, defined-benefit pension, IRA participation, and home ownership.

In [ ]:
import numpy as np
import pandas as pd
import doubleml as dml
# Load the dataset
data = pd.read_stata('/Users/prachijhamb/Downloads/sipp1991.dta')
print(f"Data shape: {data.shape}")
print("Outcome (net_tfa) and treatment (p401) summary:")
print(data[['net_tfa','p401']].describe())

The variable e401 indicates eligibility and p401 indicates participation in 401(k) plan.

In [ ]:
#Create a bar plot of the eligibility for 401(k) plans. The variable e401 has two values: 1 if the respondent is eligible for a 401(k) plan and 0 otherwise.
# Define colors explicitly
colors = ['#1f77b4', '#ff7f0e']

import matplotlib.pyplot as plt
# Set matplotlib attribute to plot the graph inline
%matplotlib inline
# Plotting
data['e401'].value_counts().plot(kind='bar', color=colors)
plt.title('Eligibility for 401(k)', fontsize=14)
plt.xlabel('e401', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

The density plots below show the comparison between these two groups. We notice that individuals eligible for a 401(k) (e401 = 1) generally have higher net total financial assets compared to individuals who are not eligible (e401 = 0). 

In [ ]:
import seaborn as sns
# Set up FacetGrid to create separate plots based on 'e401'
g = sns.FacetGrid(data, col='e401', hue='e401', palette='Set2', height=5, aspect=1.2, sharey=True)

# Add density plots within each facet
g.map(sns.kdeplot, 'net_tfa', fill=True, common_norm=False, alpha=0.5)

# Set x-axis limits to match your ggplot specification
g.set(xlim=(-20000, 150000))

# Add titles and axis labels
g.set_axis_labels('net_tfa', 'Density')
g.set_titles('e401 = {col_name}')

# Improve layout
plt.tight_layout()
plt.show()

#Estimate average potential outcomes
We compute the simple difference in means or the unconditional average predictive effect (APE) of 401(k) eligibility on accumulated assets. This measure would reflect the average treatment effect if eligibility for a 401(k) plan were randomly assigned across all individuals. 

In [ ]:
# Filter data where e401 == 1 and e401 == 0
e1 = data[data['e401'] == 1]
e0 = data[data['e401'] == 0]

# Calculate the simple difference in means for net_tfa
mean_diff_e = round(np.mean(e1['net_tfa']) - np.mean(e0['net_tfa']), 0)
print(mean_diff_e)

# Filter data based on p401
p1 = data[data['p401'] == 1]
p0 = data[data['p401'] == 0]

# Compute and round the difference in means
mean_diff_p = round(np.mean(p1['net_tfa']) - np.mean(p0['net_tfa']), 0)
print(mean_diff_p )


The unconditional APE of e401 is about 19559: individuals eligible for a 401(k) have about $19559 more in net total financial assets compared to those who are not eligible.
Among the 3682 individuals that are eligible,  2594 decided to participate in the program. The unconditional APE of p401 is about  27372: individuals who participate in a 401(k) have about $27372 more in net total financial assets compared to those who do not participate.
But, these estimates are biased since they do not account for the non-random nature of 401(k) participation decisions. 

In [ ]:
# Define the features and outcome variables
control = ['age', 'inc', 'educ', 'fsize', 'marr', 'twoearn', 'db', 'pira', 'hown']
outcome = 'net_tfa'
treatment = 'e401'

# Initialize basic DoubleMLData object
data_dml_base = dml.DoubleMLData(data, 
                                 y_col= outcome,
                                 d_cols= treatment,
                                 x_cols=control)

print(data_dml_base)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
## Step 4: Construct Polynomial Features for Flexible Model
# Start with indicator features
features = data[['marr', 'twoearn', 'db', 'pira', 'hown']].copy()

# Dictionary specifying polynomial degrees for selected variables
poly_dict = {'age': 2,
             'inc': 2,
             'educ': 2,
             'fsize': 2}

# Create polynomial features
for key, degree in poly_dict.items():
    poly = PolynomialFeatures(degree, include_bias=False)
    data_poly = poly.fit_transform(data[[key]])
    poly_feature_names = poly.get_feature_names_out([key])
    data_poly_df = pd.DataFrame(data_poly, columns=poly_feature_names, index=data.index)

    # Concatenate polynomial features
    features = pd.concat([features, data_poly_df], axis=1)

# Combine with outcome and treatment
model_data = pd.concat([data[['net_tfa', 'e401']], features], axis=1)
model_data
# Initialize the flexible DoubleMLData object
data_dml_flex = dml.DoubleMLData(model_data,
                                 y_col='net_tfa',
                                 d_cols='e401')

print(data_dml_flex)

In [ ]:
from sklearn.linear_model import LassoCV, LogisticRegressionCV
# Set random seed for reproducibility
np.random.seed(123)
## Step 3: Define Lasso Learners for DoubleML
# Lasso for outcome (regression)
lasso_g = LassoCV(cv=5, random_state=123)

# Lasso for treatment (classification)
lasso_m = LogisticRegressionCV(cv=5, penalty='l1', solver='saga', max_iter=10000, random_state=123)


## Step 4: Initialize and Fit the PLR Model
# Assuming 'data_ml' is already created as per previous step
dml_plr = dml.DoubleMLPLR(data_ml,
                          ml_g=lasso_g,
                          ml_m=lasso_m,
                          n_folds=3)

# Fit the model and store predictions
dml_plr.fit(store_predictions=True)

Linear model in Strategy 1 gives us the ATE estimate of about 5K, while Strateg 2 (DML-PLM) gives that ATE estimate of about 8K. Thus, OLS-LM understates the ATE effect by roughly 30% in relative terms.

Since DML-PLM in Strategy 2 is strictly more general than the LM in Strategy 1, we realize that the basic linear model is a pretty bad model in this example.

In summary, we've used DML-PLM to validate the simple linear model and ended up rejecting it as not very sensible. Here we can just go ahead and use DML-PLM model.
